# L7a: Multiarm Bandit Problems
In this lecture, we continue our discussion of online learning and will focus on the multiarm bandit problem. This lecture will introduce the following key concepts:
* __Bandit problems__ are a class of online learning problems where an agent, during each turn opf the game, must decide which arm (choice) to pull from a set of possible choices in order to maximize its reward. Alternatively, the agent must decide which choice to make to minimize its regret. This is similar in many ways to the multiplicative weights algorithm we discussed in the previous lecture. In a bandit problem, the agent is the aggregator, the arms (choices) are the experts, and adversary is nature.
* __Regret__ is the difference between the reward the agent would have received if it had chosen the best arm every time (in hindsight) and the reward the agent _actually_ received. Thus, the goal of the agent is to develop a _policy_ that minimizes its regret. If we only had a time machine, we could go back and make better choices.
* __Exploration vs. exploitation__ is the key tradeoff in bandit problems. The agent must decide whether to explore (try new arms) or exploit (choose the arm that has been the best so far) during each round. This is a key tradeoff in many online learning problems. If the agent always exploits, it may miss out on better arms. Hpwever, if the agent always explores, it may not take advantage of the best arm it has found so far.

The lecture notes for this week will taken from Chapter 1 of "Introduction to Multi-Armed Bandits" by Aleksandrs Slivkins. This is a great resource for learning more about bandit problems. [The book is available online](https://arxiv.org/abs/1904.07272) and is hosted on our [course GitHub repository](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-7/L7a/docs/Slivkins-Bandits-MSFT-Research-2024.pdf).

## What is a bandit problem?
A bandit problem another example of an online learning problem, i.e., a sequential descision problem in which an agent (us) makes a choice:
* If the choice at time $t$ was _good_ (compared to an adversary, i.e., nature) a positive reward is given. Alternatively, if the choice was _bad_ a penalty is given. The agent must decide which arm to pull (choice to make) in order to maximize its reward (minimize its regret).
* The memory of good (or bad) choices is stored in the agent's _policy_ which is used to make future choices. The agent must decide whether to explore (try new choices) or exploit (make a choice that has been the best so far) during each round. This is a key tradeoff in many online learning problems.

Bandit problems are used in a _wide_ range of applications. Here are a few examples:
* __Clinical Trials__: Multi-armed bandits are used to optimize the allocation of patients to different treatments, balancing the need to gather information about new treatments with the goal of minimizing patient losses. This is a real-life example of the exploration-exploitation tradeoff.
* __Financial Portfolio Design__: This approach helps in dynamically allocating investments across different assets to maximize returns while exploring new investment opportunities.
* __Adaptive Routing__: Bandit algorithms are applied to optimize routing decisions in networks, minimizing delays by balancing exploration of new routes with exploitation of known efficient paths. Networks for us could be the internet, a metabolic network, a production network, etc.
* __Recommendation systems__: Multi-armed bandits are utilized to personalize recommendations by iteratively selecting items to display, balancing the exploration of new items with the exploitation of well-performing ones. We encounter this in applications like Netflix, Amazon, etc.

For more information on the applications of bandit problems, see [the survey by Bouneffouf and Rish.](https://arxiv.org/abs/1904.10040)

## Stochastic Multi-Armed Bandits
In the stochastic multi-armed bandit problem, the agent must choose an action $a$ from the set of all possible actions $\mathcal{A}$, where $\dim\mathcal{A} = K$ during each round $t = 1,2,\dots,T$. The agent receives a reward $r_{a}$ from the environment, where $r_{a}$ is sampled from an unknown distribution $\mathcal{D}_{a}$ 

For $t = 1,2,\dots,T$:
1. _Aggregator_: The agent picks an action $a_{t} \in \mathcal{A}$. We will model this choice in many ways.
2. _Adeversary_: The agent implements action $a_{t}$ and receives a reward $r_{t}\in\left[0,1\right]$ sampled from the (unknown) distribution $\mathcal{D}_{a}\mid a = a_{t}$.
3. Agent observes $r_{t}$, but nothing else. It cannot see the distribution $\mathcal{D}_{a}$, only the _adversary_ can see this.

The agent is interested in learning the mean of the reward distribution of each arm, $\mu_{a} = \mathbb{E}\left[\mathcal{D}_{a}\right]$, by experimenting against the world (adversary). The  goal of the agent is to maximize the total reward. However, the goal of the algorithm designer is to minimize the _regret_ of the algorithm that the agent uses.

## Regret
Regret measures the difference between what could have been achieved by always making the best decision, i.e., the decision that maximizes reward (in hindsight) and what the agent actually choses to do during round. 
* __Perspective__: Regret is a property of the algorithm not the agent (which only cares about the reward). Each desicion making framework that the agent employs may lead to a different bound on regret. Thus, the goal of the algorithm designer is to minimize the regret of the algorithm that the agent uses.

__Definition__: Regret.


## Uniform Exploration
One simple approach to the multi-armed bandit problem is to explore each arm equally. This is called _uniform exploration_ or the _explore first_ algorithm. In this approach, the agent begins with a purley exploratory phase in which it pulls each arm $N$ times. After this exploration phase, the agent selects the arm that has the highest mean reward for the rest of the game. This is called the _exploitation phase_.

___

### Explore First Algorithm
The agent has $K$ arms, $\mathcal{A} = \left\{1,2,\dots,K\right\}$, and the total number of rounds is $T$. The agent uses the following algorithm to choose which arm to pull during each round:
1. _Initialization_: For each arm $a\in\mathcal{A}$, set $N_{a} = N$ where $N = (T/K)^{2/3}\cdot\mathcal{O}\left(\log{T}\right)^{1/3}$.
2. _Exploration_: Play each arm $a\in\mathcal{A}$ for $N_{a}$ rounds and record the rewards. After the exploration phase, select the arm $a^{\star}$ with the highest mean reward (break ties arbitrarily).
3. _Explotation_: Play arm $a^{\star}$ for the remaining rounds.

__Theorem__: The _expected_ regret over $T$ rounds of the _uniform exploration_ algorithm is bounded by $\mathbb{E}\left[R(T)\right]\leq{T}^{2/3}\times\mathcal{O}\left(K\cdot\log{T}\right)^{1/3}$, where $K$ is the number of arms, $T$ is the total number of rounds and $N = (T/K)^{2/3}\cdot\mathcal{O}\left(\log{T}\right)^{1/3}$ is the number of rounds in the exploration phase for each action (choice).

___

## Epsilon-Greedy Exploration
One issue with the _uniform exploration_ algorithm is that it may not be the best choice for all problems. For example, the performance in the exploration phase may be _bad_ if many of the arms have a large gap $\Delta({a})$:
* _What is this gap_? Let the (true) mean reward for each arm be $\mu(a) = \mathbb{E}\left[\mathcal{D}_{a}\right]$, where $a\in\mathcal{A}$. The _best_ mean reward over the actions is $\mu^{\star} = \max_{a\in\mathcal{A}}\mu_{a}$. Then, the gap $\Delta({a}) = \mu^{\star} - \mu(a)$ is the difference between the mean reward of the best arm and the mean reward of arm $a$. If the gap is _large_, the agent may miss out on a lot of reward by exploring each arm equally.

In cases of a large gap, it may be better to spread out (and interleave) the exploration and explotation phases the arms. This is the idea behind the _epsilon-greedy_ algorithm. In this algorithm, the agent chooses the best arm with probability $1-\epsilon$ and chooses a random arm with probability $\epsilon$. This allows the agent to explore the arms more evenly and may lead to better performance in cases where the gap is large.

___
### Epsilon-Greedy Algorithm
The agent has $K$ arms (choices), $\mathcal{A} = \left\{1,2,\dots,K\right\}$, and the total number of rounds is $T$. The agent uses the following algorithm to choose which arm to pull (which action to take) during each round:

For $t = 1,2,\dots,T$:
1. _Initialize_: Roll a random number $p\in\left[0,1\right]$ and compute the threshold $\epsilon_{t}\sim{t}^{-1/3}$.
2. _Exploration_: If $p\leq\epsilon_{t}$, choose a random (uniform) arm $a_{t}\in\mathcal{A}$. Execute the action $a_{t}$ and recieve a reward $r_{t}$ from the _adversary_ (nature). 
3. _Exploitation_: Else if $p>\epsilon_{t}$, choose action $a^{\star}$ (action with the highest average reward so far, the greedy choice). Execute the action $a^{\star}_{t}$ and recieve a reward $r_{t}$ from the _adversary_ (nature).
4. Update list of rewards for $a_{t}\in\mathcal{A}$

__Theorem__: The epsilon-greedy algowithm with exploration probability $\epsilon_{t}={t^{-1/3}}\cdot\left(K\cdot\log(t)\right)^{1/3}$ achives a regret bound of $\mathbb{E}\left[R(t)\right]\leq{t}^{2/3}\cdot\left(K\cdot\log(t)\right)^{1/3}$ for each round $t$.
___

## Optimism under uncertainty
Let's consider our final approach for solving bandit problems, namely the optimism under uncertainty algorithm. The key assumption of this approach:
* __Assumption__: each arm is as good as it can possibly be given the observations so far, and choose the best arm based on these optimistic estimates. This intuition leads to the `UCB1` algorithm.

Given a history of rewards and a number of pulls for each arm, the `UCB1` algorithm calculates an _upper confidence bound_ (UCB) which it uses to decide which arm to pull. 

__Definition__: Upper Confidence Bound (UCB). During each round $t =1,2, \dots, T$ the `UCB1` algorithm maximizes the sum $\bar{\mu}(a)+U(a,t)$ where $\bar{\mu}(a)$ is the _estimated_ mean return of arm $a\in\mathcal{A}$ at time $t$ and $U(a,t)$ is the _upper confidence bound_ of arm $a\in\mathcal{A}$ at time $t$:
$$
\begin{align*}
U(a,t) = \sqrt{\frac{2\log(t)}{N_{a}}}
\end{align*}
$$
where $N_{a}$ is the number of times that arm $a\in\mathcal{A}$ has been pulled up to time $t$. The `UCB1` algorithm chooses the arm $a^{\star}$ that maximizes the sum $\bar{\mu}(a)+U(a,t)$ during each round $t$. The `UCB1` algorithm was originally proposed by Auer, Cesa-Bianchi, and Fischer in 2002:
* [Peter Auer, Nicolo Cesa-Bianchi, and Paul Fischer. Finite-time analysis of the multiarmed bandit problem. Machine
Learning, 47(2-3):235–256, 2002a.](https://people.eecs.berkeley.edu/~russell/classes/cs294/s11/readings/Auer+al:2002.pdf)

___
### UCB1 Algorithm
The agent has $K$ arms (choices), $\mathcal{A} = \left\{1,2,\dots,K\right\}$, and a total number of rounds is $T\gg{K}$.

_Initialization_: Pull each arm $a\in\mathcal{A}$ once and record the rewards. For each arm $a\in\mathcal{A}$, set $N_{a} = 1$ and $\bar{\mu}(a) = r_{a}$.

For rounds $t = K+1,K+2,\dots,T$:
1. Compute the upper confidence bound $U(a,t)$ for each arm $a\in\mathcal{A}$.
2. Choose the best arm $a^{\star} = \text{arg}\max\,\left\{\bar{\mu}(a)+U(a,t)\mid\,a\in\mathcal{A}\right\}$ at time $t$.
3. Execute the action $a^{\star}$ and recieve a reward $r_{t}$ from the _adversary_ (nature).
4. Update the estimated mean reward $\bar{\mu}(a^{\star})$ and the number of pulls $N_{a^{\star}}$

__Theorem__: The `UCB1` algorithm achieves a regret bound for $K$ arms of $\mathbb{E}\left[R(t)\right]\leq\mathcal{O}\left(\sqrt{KT\cdot\log(T)}\right)$ 
over $T$ rounds.
___

# Today?
That's a wrap! What are some of the interesting things we discussed today?